In [20]:
from product.stock_category import StockCategory
from database.market import Market
from database.sec import SEC
from datetime import datetime, timedelta
from processor.processor import Processor as p
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [22]:
factors = ['AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'Assets',
 'AssetsCurrent',
 'CashAndCashEquivalentsAtCarryingValue',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'EntityCommonStockSharesOutstanding',
 'Goodwill',
 'IncomeTaxExpenseBenefit',
 'LiabilitiesAndStockholdersEquity',
 'LiabilitiesCurrent',
 'OtherAssetsNoncurrent',
 'PropertyPlantAndEquipmentNet',
 'RetainedEarningsAccumulatedDeficit',
 'filed',
 'StockholdersEquity',
 'NetIncomeLoss']

In [2]:
market = Market()
sec = SEC()

## Currently not modeling stock category due to low actual accuracy

In [17]:
params = {}
product = StockCategory(params)
product.subscribe()
# sim = product.create_sim()
# product.load()
# # product.transformed

In [3]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [11]:
centralized = ['assets',
 'liabilitiesandstockholdersequity',
 'netcashprovidedbyusedinfinancingactivities',
 'netcashprovidedbyusedininvestingactivities',"date","ticker"]

In [7]:
sec.connect()
market.connect()
ticker = "MMM"
cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
filing = sec.retrieve_filing_data(cik)
filing["ticker"] = ticker
filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
filing = p.column_date_processing(filing)
centralized = [x[:-1] if x[-1] == "s" else x for x in list(filing.columns)]
for ticker in tqdm(sp5["Symbol"]):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        filing["ticker"] = ticker
        filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
        filing = p.column_date_processing(filing)
        current_columns = [x[:-1] if x[-1] == "s" else x for x in list(filing.columns)]
        if filing.index.size > 40:
            initial = [x for x in current_columns if x in centralized]
            centralized = [x for x in centralized if x in initial]
    except Exception as e:
        print(ticker, str(e))
sec.disconnect()
market.disconnect()

 39%|████████████████▍                         | 197/505 [01:02<02:15,  2.28it/s]

FRC 'filed'


 83%|██████████████████████████████████▋       | 417/505 [02:09<00:36,  2.44it/s]

SBNY 'filed'


100%|██████████████████████████████████████████| 505/505 [02:36<00:00,  3.23it/s]


In [36]:
sec.connect()
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        filing["ticker"] = ticker
        filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
        filing = p.column_date_processing(filing)
        if filing.index.size > 40:
            market.store("unified_financials",filing[centralized].fillna(method="ffill"))
    except Exception as e:
        print(ticker, str(e))
sec.disconnect()
market.disconnect()

 39%|████████████████▍                         | 197/505 [01:02<02:14,  2.28it/s]

FRC 'filed'


 83%|██████████████████████████████████▋       | 417/505 [02:10<00:35,  2.45it/s]

SBNY 'filed'


100%|██████████████████████████████████████████| 505/505 [02:36<00:00,  3.23it/s]


In [37]:
market.connect()
data = market.retrieve("unified_financials")
market.disconnect()

In [38]:
data["year"] = [x.year for x in data["date"]]
data["quarter"] = [x.quarter for x in data["date"]]

In [39]:
data

,assets,liabilitiesandstockholdersequity,netcashprovidedbyusedinfinancingactivities,netcashprovidedbyusedininvestingactivities,date,ticker,year,quarter
0,2.594400e+10,2.594400e+10,-1.122000e+09,-9.210000e+08,2009-07-31,MMM,2009,3
1,2.594400e+10,2.594400e+10,-1.122000e+09,-9.210000e+08,2009-07-31,MMM,2009,3
2,2.652150e+10,2.652150e+10,-2.099333e+09,-1.832667e+09,2010-02-16,MMM,2010,1
3,2.652150e+10,2.652150e+10,-2.099333e+09,-1.832667e+09,2010-02-16,MMM,2010,1
4,2.763600e+10,2.763600e+10,-5.910000e+08,-5.025000e+08,2010-05-05,MMM,2010,2
...,...,...,...,...,...,...,...,...
22505,7.280950e+10,7.280950e+10,3.593500e+09,-4.008000e+09,2020-08-05,ZION,2020,3
22506,7.376450e+10,7.376450e+10,4.471500e+09,-4.892000e+09,2020-11-04,ZION,2020,4
22507,7.532550e+10,7.532550e+10,3.979333e+09,-4.845000e+09,2021-02-25,ZION,2021,1
22508,8.330000e+10,8.330000e+10,2.772000e+09,-3.061500e+09,2021-05-06,ZION,2021,2


In [48]:
start = data["year"].min()
end = data["year"].max()
training_years = 4
product.db.connect()
the_cats = []
for year in tqdm(range(2014,2022)):
    try:
#         pca_model = PCA(n_components=3)
#         pca_model.fit(test_set)
#         new_set = pca_model.transform(test_set)
#         visuals = pd.DataFrame(new_set,columns=["pca1","pca2","pca3"])
        training_set = data[(data["year"] < year) & (data["year"] > year - training_years)].copy().dropna()
        prediction_set = data[(data["year"]==year)].copy().dropna()
        non_relevant = ["date","quarter","ticker","year","training_years"]
        factors = [x for x in training_set.columns if x not in non_relevant]
        kmeans = KMeans(n_clusters=10,random_state=42).fit(training_set[factors])
        cats = kmeans.predict(prediction_set[factors])
        prediction_set["4_classification"] = cats
        the_cats.append(prediction_set[['year','quarter','ticker','4_classification']])
    except Exception as e:
        print(str(e))
product.db.disconnect()

100%|██████████████████████████████████████████████| 8/8 [00:00<00:00, 10.45it/s]


In [50]:
stuff = pd.concat(the_cats)
product.db.connect()
sim = product.db.retrieve("sim")
product.db.disconnect()

In [51]:
new_sim = sim.merge(stuff,how="left",on=["year","quarter","ticker"])

In [53]:
product.db.connect()
product.db.store("sim",new_sim)
product.db.disconnect()